In [1]:
%cd ..

/home/isaiah/code/NestedGraphs


In [2]:
import glob 
import pickle
import torch 

from utils.graph_utils import propagate_labels

FEATS = 'feature_extraction/data/features/'
GRAPHS = 'inputs/Sept23/mal/'

In [4]:
embs = []
labels = []
missing = []

for gf in glob.glob(GRAPHS+'*.pkl'):
    with open(gf, 'rb') as f:
        g = pickle.load(f)

    procs = (g.x[:,0] == 1).nonzero()
    y = propagate_labels(g, 23)

    uuids = []
    for p in procs:
        uuids.append(g.get(p)['uuid'])

    # This will be updated so the dict doesn't have to be made
    # every time
    mapping,feats = torch.load(FEATS+'%d.pkl' % g.gid)
    mapping = {m:i for i,m in enumerate(mapping)}

    for i,uuid in enumerate(uuids):
        if fid := mapping.get(uuid):
            embs.append(feats[fid])
        else:
            missing.append((uuid,y[i]))

    labels.append(y)

embs = torch.stack(embs)
labels = torch.cat(labels, dim=0)

print(embs.size(),labels.size())
print(embs)

503 tensor([0])
205 tensor([0])
321 tensor([0])
609 tensor([0])
255 tensor([0])
419 tensor([0])
355 tensor([0])
104 tensor([0])
955 tensor([0])
874 tensor([0])
660 tensor([0])
771 tensor([0])
559 tensor([0])
462 tensor([0])
201 tensor([0])
torch.Size([66810, 128]) torch.Size([434180])
tensor([[ 0.0128, -0.2468,  0.0728,  ...,  0.1410,  0.1636,  0.1073],
        [-0.1379, -0.2820, -0.1315,  ..., -0.1061, -0.3675,  0.1687],
        [ 0.0706, -0.0543,  0.0765,  ..., -0.0588, -0.0171,  0.0179],
        ...,
        [-0.0373, -0.3295, -0.3702,  ..., -0.5143, -0.9885,  0.2540],
        [-0.0522, -0.1024,  0.1760,  ..., -0.0220,  0.0159, -0.0022],
        [-0.1379, -0.2820, -0.1315,  ..., -0.1061, -0.3675,  0.1687]],
       device='cuda:3')


: 